In [5]:
from quote_prediction.data import ExplicitQuoteCorpus, Novel
import numpy as np 
import tqdm 
import torch
from sklearn.metrics import pairwise_distances
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt 
from matplotlib.lines import Line2D
import matplotlib.cm as mplcm
import matplotlib.colors as colors
from collections import defaultdict
from nltk import word_tokenize
import re 
import spacy
from sklearn.metrics import roc_auc_score
from sentence_transformers import SentenceTransformer
from sentence_transformers import util 
import torch.nn.functional as F 
from transformers import AutoTokenizer
from transformers import RobertaForCausalLM, AutoConfig, AutoModel, RobertaModel, BertModel, AutoModelForSequenceClassification
from transformers.trainer import Trainer
from sklearn.metrics import ndcg_score

In [ ]:
!python main.py --model drama_luar

2024-05-08 12:06:26,981 - Starting experiment ALL with query size >= 5
2024-05-08 12:06:26,981 - Loading data...
2024-05-08 12:06:32,951 - Loaded quotes of 28 novels
2024-05-08 12:06:32,951 - Starting sampling .....
NOVEL: AHandfulOfDust, ID 0 - # of queries: 25 - avg # of targets: 16.0 - avg # of quote targets: 1388.0 - avg query length: 61.8 - # of speakers in novel 17 - Percent Active Speakers 0.588
NOVEL: APassageToIndia, ID 1 - # of queries: 88 - avg # of targets: 18.0 - avg # of quote targets: 1904.2 - avg query length: 21.5 - # of speakers in novel 18 - Percent Active Speakers 1.000
NOVEL: ARoomWithAView, ID 2 - # of queries: 67 - avg # of targets: 10.0 - avg # of quote targets: 1474.4 - avg query length: 22.5 - # of speakers in novel 10 - Percent Active Speakers 1.000
NOVEL: AlicesAdventuresInWonderland, ID 3 - # of queries: 29 - avg # of targets: 11.0 - avg # of quote targets: 541.5 - avg query length: 19.0 - # of speakers in novel 11 - Percent Active Speakers 0.909
NOVEL: Ann

In [6]:
from metrics import *
from utils import luar_tokenize, get_model
model, tokenizer = get_model("luar")

/usr/local/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

...

model.py:   0%|          | 0.00/8.33k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/rrivera1849/LUAR-MUD:
- model.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/330M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

In [7]:
model, tokenizer = get_model("drama_luar")

In [8]:
model = model.to(torch.device("cuda:0"))

In [10]:
model

Transformer(
  (transformer): MPNetModel(
    (embeddings): MPNetEmbeddings(
      (word_embeddings): Embedding(30527, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): MPNetEncoder(
      (layer): ModuleList(
        (0): MPNetLayer(
          (attention): MPNetAttention(
            (attn): MPNetSelfAttention(
              (q): Linear(in_features=768, out_features=768, bias=True)
              (k): Linear(in_features=768, out_features=768, bias=True)
              (v): Linear(in_features=768, out_features=768, bias=True)
              (o): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          

In [2]:
corpus = ExplicitQuoteCorpus("/data/datasets/project-dialogism-novel-corpus/data/")

In [43]:
percentage = []
count_no_exp = 0
no_exp_major = 0 
total_count = 0

for n in corpus : 
    d = {}
    exp = np.asarray(n["is_explicit"])
    sid =  np.asarray(n["speaker_id"])
    for id in np.unique(n["speaker_id"]) : 
        quotes = np.where(sid == id)[0]
        
        if sum(exp[quotes]) >= 5 : 
            d[id] = sum(exp[quotes]) / len(quotes) 
        else : 
            d[id] = 0 
            count_no_exp += 1
            if n["is_major"][id] == 1 : 
                no_exp_major += 1
    percentage.append(np.mean(list(d.values())))

        
    total_count += len(d)

In [46]:
no_exp_major / count_no_exp

0.17142857142857143

In [41]:
count_no_exp / total_count

0.11290322580645161

In [4]:
min_utterances_for_query = 1
iterator = [
    (
        corpus.chapterwise_AV_samples(
            min_utterances_for_anchor=min_utterances_for_query
        ),
        "chapterwise",
    ),
    (
        corpus.chapterwise_AV_samples(
            train_with_explicit=True,
            test_without_explicit=True,
            min_utterances_for_anchor=min_utterances_for_query,
        ),
        "explicit_to_other",
    ),
    (
        corpus.chapterwise_AV_samples(
            train_with_explicit=True,
            test_without_explicit=False,
            min_utterances_for_anchor=min_utterances_for_query,
        ),
        "explicit_to_all",
    ),
]
ns = [1, 5, 10, 20, 50, 100]
pairs = []
for n in ns:
    print("*" * 10 + f" PROCESSING # UTTERANCES {n }" + "*" * 10)
    p = corpus.utterances_AV_samples(n_utterances=n, test_percentage=0.5)
    pairs.append(p)
iterator.append((pairs, "reading_order"))

NOVEL: AHandfulOfDust, ID 0 - # of queries: 30 - avg # of targets: 15.8 - avg # of quote targets: 1419.1 - avg query length: 52.0 - # of speakers in novel 17 - Percent Active Speakers 0.588
NOVEL: APassageToIndia, ID 1 - # of queries: 115 - avg # of targets: 18.0 - avg # of quote targets: 1910.8 - avg query length: 17.2 - # of speakers in novel 18 - Percent Active Speakers 1.000
NOVEL: ARoomWithAView, ID 2 - # of queries: 84 - avg # of targets: 10.0 - avg # of quote targets: 1475.4 - avg query length: 18.5 - # of speakers in novel 10 - Percent Active Speakers 1.000
NOVEL: AlicesAdventuresInWonderland, ID 3 - # of queries: 36 - avg # of targets: 11.0 - avg # of quote targets: 543.6 - avg query length: 16.1 - # of speakers in novel 11 - Percent Active Speakers 0.909
NOVEL: AnneOfGreenGables, ID 4 - # of queries: 106 - avg # of targets: 5.0 - avg # of quote targets: 1066.0 - avg query length: 10.2 - # of speakers in novel 5 - Percent Active Speakers 1.000
NOVEL: DaisyMiller, ID 5 - # of q

In [ ]:
[i.]

In [4]:
sizes = []
for quote in corpus["quotes"] : 
    for q in quote : 
        v = tokenizer(
        q,
        return_tensors="pt",
    )
        
        sizes.append(v.input_ids.size(1))

Token indices sequence length is longer than the specified maximum sequence length for this model (681 > 512). Running this sequence through the model will result in indexing errors


In [5]:
sum(sizes) / len(sizes)

35.657857205227465

In [13]:
v["input_ids"].size()

torch.Size([1561, 1, 128])

In [4]:
[(n, i.author, i.name, i.genre, i.narrative_person) for n,i in enumerate(corpus)]

[(0, 'Evelyn Waugh', 'AHandfulOfDust', 'literary', 3.0),
 (1, 'Edward Morgan Forster', 'APassageToIndia', 'literary', 3.0),
 (2, 'Edward Morgan Forster', 'ARoomWithAView', 'literary', 3.0),
 (3, 'Lewis Carroll', 'AlicesAdventuresInWonderland', 'children', 3.0),
 (4, 'Lucy Maud Montgomery', 'AnneOfGreenGables', 'children', 3.0),
 (5, 'Henry James', 'DaisyMiller', 'literary', 1.0),
 (6, 'Jane Austen', 'Emma', 'literary', 3.0),
 (7, 'Charles Dickens', 'HardTimes', 'literary', 3.0),
 (8, 'Edward Morgan Forster', 'HowardsEnd', 'literary', 3.0),
 (9, 'Jane Austen', 'MansfieldPark', 'literary', 3.0),
 (10, 'Virginia Woolf', 'NightAndDay', 'literary', 3.0),
 (11, 'Jane Austen', 'NorthangerAbbey', 'literary', 3.0),
 (12, 'Charles Dickens', 'OliverTwist', 'literary', 3.0),
 (13, 'Jane Austen', 'Persuasion', 'literary', 3.0),
 (14, 'Jane Austen', 'PrideAndPrejudice', 'literary', 3.0),
 (15, 'Jane Austen', 'SenseAndSensibility', 'literary', 3.0),
 (16, 'Edith Wharton', 'TheAgeOfInnocence', 'litera

In [10]:
authors = [i.genre for n,i in enumerate(corpus)]
import numpy as np 
np.unique(authors, return_counts=True)

(array(['children', 'crime', 'literary', 'scifi'], dtype='<U8'),
 array([ 3,  2, 22,  1]))

In [ ]:
[(n, i.genre) for n,i in enumerate(corpus)]

In [37]:
sum(corpus[24]["is_explicit"]) / len(corpus[24]["is_explicit"])

0.12859884836852206

In [43]:
expl = corpus[24].quote_table[corpus[24].quote_table["is_explicit"]==1]
expl[expl["Character ID"]==15]

,quote_id,quoteID,quoteText,subQuotationList,quoteByteSpans,speaker,addressees,quoteType,referringExpression,mentionTextsList,mentionSpansList,mentionEntitiesList,is_explicit,Character ID,role
182,214,Q214,"You 'd just as well tell Kitty now,\nma, for s...","[You 'd just as well tell Kitty now,\nma, for ...","[[60648, 60728]]",Joe Hamilton,"[Kitty Hamilton, Fannie Hamilton]",Explicit,Joe spoke,"[[You, Kitty, ma, she]]","[[[60648, 60651], [60673, 60678], [60684, 6068...","[[[Fannie Hamilton], [Kitty Hamilton], [Fannie...",1,15,major
187,221,Q221,"I 'd like to cut the heart out of a few of 'em,","[I 'd like to cut the heart out of a few of 'em,]","[[61831, 61878]]",Joe Hamilton,"[Kitty Hamilton, Fannie Hamilton]",Explicit,said Joe,[[]],[[]],[[]],1,15,major
195,231,Q231,"Let 's go to New York,","[Let 's go to New York,]","[[63727, 63749]]",Joe Hamilton,"[Kitty Hamilton, Fannie Hamilton]",Explicit,said Joe,[[s]],"[[[63732, 63733]]]","[[[Fannie Hamilton, Joe Hamilton, Kitty Hamilt...",1,15,major
204,243,Q243,"Great place,","[Great place,]","[[75048, 75060]]",Joe Hamilton,"[Kitty Hamilton, Mr. Thomas, Mrs. Jones, Fanni...",Explicit,said Joe,[[]],[[]],[[]],1,15,major
245,297,Q297,"No, no, this is on me,","[No, no, this is on me,]","[[94040, 94062]]",Joe Hamilton,[Mr. Thomas],Explicit,cried Joe,[[]],[[]],[[]],1,15,major
247,299,Q299,"Happy to know you, Mr.\nWilliams.","[Happy to know you, Mr.\nWilliams.]","[[94831, 94863]]",Joe Hamilton,"[Mr. Thomas, Sadness, Mr. Turner]",Explicit,Joes,"[[you, Mr.\nWilliams]]","[[[94845, 94848], [94850, 94862]]]","[[[Sadness], [Sadness]]]",1,15,major
252,304,Q304,"I cert'n'y will,","[I cert'n'y will,]","[[95712, 95728]]",Joe Hamilton,"[Mr. Thomas, Sadness, Mr. Turner]",Explicit,was Joes opportune remark,[[]],[[]],[[]],1,15,major
265,319,Q319,"Oh, she did n't ask for me,","[Oh, she did n't ask for me,]","[[106283, 106310]]",Joe Hamilton,[Mr. Thomas],Explicit,cried Joe,[[she]],"[[[106287, 106290]]]",[[[Hattie Sterling]]],1,15,major
272,326,Q326,"Goo'-night, I be a' ri'. Goo'-night.","[Goo'-night,, I be a' ri'. Goo'-night.]","[[107401, 107412], [107434, 107458]]",Joe Hamilton,"[Mr. Thomas, Hattie Sterling]",Explicit,said Joe,"[[], []]","[[], []]","[[], []]",1,15,major
299,365,Q365,"There, there!\nsee what you 've done with your...","[There, there!\nsee what you 've done with you...","[[118174, 118239]]",Joe Hamilton,"[Kit Hamilton, Fannie Hamilton]",Explicit,Joe burst out,"[[you, your]]","[[[118197, 118200], [118215, 118219]]]","[[[Fannie Hamilton], [Fannie Hamilton]]]",1,15,major


In [11]:
pairs = corpus[0].chapterwise_AV_samples(True)

In [18]:
pairs[2][1]

[66, 77, 89, 104, 123, 194]

In [12]:
v = sum([i.tolist() for i in corpus[0].quote_id_by_chapter],[])
max(v), len(corpus[0]["is_explicit"])

(1884, 1885)

In [6]:
negs = [pairs[i][2] for i in range(len(pairs))]
anc = [pairs[i][1] for i in range(len(pairs))]

sids = [pairs[i][0] for i in range(len(pairs))]

for sid, neg in zip(sids, negs) : 
    for speaker_id, nn in neg.items() : 
        print(speaker_id, nn)
    print("\n\n\n")

5 [1614, 1615, 1616, 1617, 1618, 1619, 1620, 1621, 1622, 1623, 1625, 1627, 1629, 1631, 1633, 1635, 1637, 1656, 1657, 1658, 1659, 1713, 1715, 1717, 1719, 1721, 1723, 1725, 1727, 1729]
7 [1574, 1575, 1576, 1577, 1578, 1624, 1626, 1628, 1630, 1632, 1634, 1636, 1638, 1640, 1642, 1644, 1646, 1711, 1712, 1714, 1716, 1718, 1720, 1722, 1724, 1726, 1728, 1730, 1771, 1772, 1773, 1774, 1775, 1776, 1777, 1778, 1779]
9 [1507, 1508, 1510, 1512, 1514, 1516, 1517, 1519, 1520, 1521, 1522, 1523, 1525, 1542, 1570, 1571, 1572, 1573, 1579, 1580, 1581, 1583, 1585, 1587, 1589, 1591, 1593, 1595, 1597, 1598, 1599, 1601, 1604, 1606, 1607, 1609, 1611, 1613, 1647, 1648, 1649, 1651, 1653, 1655, 1660, 1662, 1663, 1665, 1667, 1669, 1670, 1671, 1672, 1674, 1676, 1678, 1679, 1681, 1683, 1685, 1687, 1688, 1690, 1691, 1693, 1694, 1695, 1696, 1697, 1699, 1701, 1703, 1705, 1707, 1708, 1710, 1731, 1733, 1735, 1736, 1738, 1740, 1742, 1747, 1749, 1751, 1753, 1755, 1756, 1758, 1760, 1761, 1763, 1765, 1767, 1769]
10 [1639, 164

In [15]:
sids

[5, 7, 10, 35]

# Chapterwise Author Verification

Here, sample pairs are create as follows:

- A positive pair is (quotes from speaker A in chapter C ; quotes from speaker A in all other chapters)
- A negative pair is (quotes from speaker A in chapter C ; quotes from speaker B in all other chapters)

If models are able to discriminate against positive and negative examples, it means that quotes of characters within a chapter are more self-similar than the quotes of other characters 

In [4]:
pairs = corpus.chapterwise_AV_samples()

NOVEL: AHandfulOfDust, ID 0 Percent Active Speakers 0.588
NOVEL: APassageToIndia, ID 1 Percent Active Speakers 1.000
NOVEL: ARoomWithAView, ID 2 Percent Active Speakers 1.000
NOVEL: AlicesAdventuresInWonderland, ID 3 Percent Active Speakers 0.909
NOVEL: AnneOfGreenGables, ID 4 Percent Active Speakers 1.000
NOVEL: DaisyMiller, ID 5 Percent Active Speakers 0.833
NOVEL: Emma, ID 6 Percent Active Speakers 1.000
NOVEL: HardTimes, ID 7 Percent Active Speakers 0.929
NOVEL: HowardsEnd, ID 8 Percent Active Speakers 1.000
NOVEL: MansfieldPark, ID 9 Percent Active Speakers 1.000
NOVEL: NightAndDay, ID 10 Percent Active Speakers 0.846
NOVEL: NorthangerAbbey, ID 11 Percent Active Speakers 1.000
NOVEL: OliverTwist, ID 12 Percent Active Speakers 0.929
NOVEL: Persuasion, ID 13 Percent Active Speakers 1.000
NOVEL: PrideAndPrejudice, ID 14 Percent Active Speakers 1.000
NOVEL: SenseAndSensibility, ID 15 Percent Active Speakers 1.000
NOVEL: TheAgeOfInnocence, ID 16 Percent Active Speakers 0.917
NOVEL: The

## SEMANTICS

In [ ]:
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
quote_embeddings = []

for quotes in tqdm.tqdm(corpus["quotes"]) : 
    quote_embeddings.append(model.encode(quotes, device=torch.device("cuda:0"), normalize_embeddings=True, convert_to_numpy=False, convert_to_tensor=True).cpu())

### Character - Character

In [ ]:
scores = score(quote_embeddings, pairs)

In [ ]:
plt.figure(figsize=(12,6))
plt.bar([f"novel_{i}" for i in range(len(aucs))], aucs)
x = plt.xticks(rotation = 40, ha="right")
plt.ylim(0.4)
plt.title("Semantics AUCs")

## STEL

In [ ]:
model = SentenceTransformer('AnnaWegmann/Style-Embedding')
quote_embeddings = []

for quotes in tqdm.tqdm(corpus["quotes"]) : 
    quote_embeddings.append(model.encode(quotes, device=torch.device("cuda:0"), normalize_embeddings=True, convert_to_numpy=False, convert_to_tensor=True))

In [ ]:
overall_auc, aucs, recalls = score(quote_embeddings, pairs)

In [ ]:
plt.figure(figsize=(12,6))
plt.bar([f"novel_{i}" for i in range(len(aucs))], aucs)
x = plt.xticks(rotation = 40, ha="right")
plt.ylim(0.4)
plt.title("STEL (Stylo) AUCs")

## Emotions

In [5]:


tokenizer = AutoTokenizer.from_pretrained('SamLowe/roberta-base-go_emotions')
config = AutoConfig.from_pretrained("SamLowe/roberta-base-go_emotions")
model = AutoModelForSequenceClassification.from_pretrained("SamLowe/roberta-base-go_emotions", config=config).to("cuda:0")



quote_embeddings = []

batch_size = 16
with torch.no_grad() : 
    for idx, quotes in tqdm.tqdm(enumerate(corpus["quotes"])) : 
        last_h = []
        
        model.eval()
        for idx in range(0,len(quotes), batch_size) : 
            tokens = tokenizer(quotes[idx:idx+batch_size], return_tensors="pt", truncation=True, max_length=512, padding=True)
            u = model(**tokens.to("cuda:0"), return_dict=True, output_hidden_states = True)
            last_h.append(u.hidden_states[-1][:,0].detach().cpu())
        quote_embeddings.append(F.normalize(torch.cat(last_h), dim = -1))

7it [00:26,  3.83s/it]


KeyboardInterrupt: 

In [6]:
quote_embeddings[0].size()

torch.Size([1885, 768])

In [7]:
corpus["quotes"][0].__len__()

1885

In [ ]:
overall_auc, aucs, recalls = score(quote_embeddings, pairs)

In [ ]:
plt.figure(figsize=(12,6))
plt.bar([f"novel_{i}" for i in range(len(aucs))], aucs)
x = plt.xticks(rotation = 40, ha="right")
plt.ylim(0.4)
plt.title("Emotions AUCs")

## LUAR

In [ ]:
!pip install einops

In [ ]:
#device = "cpu"
tokenizer = AutoTokenizer.from_pretrained("rrivera1849/LUAR-MUD", trust_remote_code=True)
model = AutoModel.from_pretrained("rrivera1849/LUAR-MUD", trust_remote_code=True).to("cuda:0")
def tokenize(quotes) : 
    tokens = tokenizer(quotes, max_length = 32, return_tensors = "pt", truncation=True, padding="max_length")
    tokens["input_ids"] = tokens["input_ids"].reshape(1, -1, 32)
    tokens["attention_mask"] = tokens["attention_mask"].reshape(1, -1, 32)
    
    return tokens

In [ ]:
labels = []
preds = []
aucs = []
R1 = []
R3 = []
R8 = []

model.eval()
with torch.no_grad() : 
    for nidx, novel_pairs in enumerate(pairs) : 
        
        labels_n = []
        preds_n = []
        r_1, r_3, r_8 = [], [], []
        quotes = np.asarray( corpus["quotes"][nidx])
        for u, (speaker_id, anchor, neg_and_pos) in enumerate(novel_pairs) :
             
            q1 =quotes[anchor]
            tokens = tokenize(q1.tolist())
            ep1 = F.normalize( model(**tokens.to("cuda:0")), dim =1 ).cpu()
            
            q2 = quotes[neg_and_pos[speaker_id]]
            tokens = tokenize(q2.tolist())
            ep2 = F.normalize( model(**tokens.to("cuda:0")), dim =1 ).cpu()
            
            embeddings = [ep2]
            for sid, quote_ids in neg_and_pos.items() :
                if sid != speaker_id  : 
                    neg = quotes[quote_ids]
                    tokens = tokenize(neg.tolist())
                    eneg = F.normalize( model(**tokens.to("cuda:0")), dim =1 )                
                    embeddings.append(eneg.cpu())
                    
            sims = util.dot_score(ep1[0], torch.cat(embeddings, dim = 0))[0]
            sorted_idx = torch.argsort(sims)
            r_1.append(1 if 0 in sorted_idx[-1:] else 0 )
            r_3.append(1 if 0 in sorted_idx[-3:] else 0 )
            r_8.append(1 if 0 in sorted_idx[-8:] else 0 )

            labels_n.extend([1] + [0] * (len(sims) - 1))
            preds_n.extend(sims.tolist())    
            
        R8.append(sum(r_8) / len(novel_pairs))
        R1.append(sum(r_1) / len(novel_pairs))
        R3.append(sum(r_3) / len(novel_pairs))

        aucs.append(roc_auc_score(labels_n, preds_n))
        labels.extend(labels_n)
        preds.extend(preds_n)
        print(f"NOVEL {nidx} ---- Recall@{1} {R1[-1]:0.3f} | Recall@{3} {R3[-1]:0.3f} | Recall@{8} {R8[-1]:0.3f} | AUC {aucs[-1]:0.3f}")

    overall_auc = roc_auc_score(labels, preds)
    print("")
    print(f"Recall@{1} {np.mean(R1):0.3f} | Recall@{3} {np.mean(R3):0.3f} | Recall@{8} {np.mean(R8):0.3f} | AUC (average novels) {np.mean(aucs):0.3f} | Overall AUC {overall_auc:0.3f}")

In [ ]:
plt.figure(figsize=(12,6))
plt.bar([f"novel_{i}" for i in range(len(aucs))], aucs)
x = plt.xticks(rotation = 40, ha="right")
plt.ylim(0.4)
plt.title("LUAR (Reddit) AUCs")

# Chapterwise Author Verification (Quote by quote)

## Semantics

In [ ]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')
quote_embeddings = []

for quotes in tqdm.tqdm(corpus["quotes"]) : 
    quote_embeddings.append(model.encode(quotes, device=torch.device("cuda:0"), normalize_embeddings=True, convert_to_numpy=False, convert_to_tensor=True))

In [ ]:
scores = score_quote_by_quote(quote_embeddings, pairs, simulate_randomness=True)

## STEL

In [ ]:
model = SentenceTransformer('AnnaWegmann/Style-Embedding')
quote_embeddings = []

for quotes in tqdm.tqdm(corpus["quotes"]) : 
    quote_embeddings.append(model.encode(quotes, device=torch.device("cuda:0"), normalize_embeddings=True, convert_to_numpy=False, convert_to_tensor=True))

In [ ]:
scores = score_quote_by_quote(quote_embeddings, pairs, simulate_randomness=True)

## Emotions

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('SamLowe/roberta-base-go_emotions')
config = AutoConfig.from_pretrained("SamLowe/roberta-base-go_emotions")
model = AutoModelForSequenceClassification.from_pretrained("SamLowe/roberta-base-go_emotions", config=config).to("cuda:0")


quote_embeddings = []

batch_size = 16
with torch.no_grad() : 
    for idx, quotes in tqdm.tqdm(enumerate(corpus["quotes"])) : 
        last_h = []
        
        model.eval()
        for idx in range(0,len(quotes), batch_size) : 
            tokens = tokenizer(quotes[idx:idx+batch_size], return_tensors="pt", truncation=True, max_length=512, padding=True)
            u = model(**tokens.to("cuda:0"), return_dict=True, output_hidden_states = True)
            last_h.append(u.hidden_states[-1][:,0].detach().cpu())
        quote_embeddings.append(F.normalize(torch.cat(last_h), dim = -1))

In [ ]:
scores = score_quote_by_quote(quote_embeddings, pairs)

## LUAR

In [ ]:
!pip install einops

In [ ]:
#device = "cpu"
tokenizer = AutoTokenizer.from_pretrained("rrivera1849/LUAR-MUD", trust_remote_code=True)
model = AutoModel.from_pretrained("rrivera1849/LUAR-MUD", trust_remote_code=True).to("cuda:0")


def tokenize(quotes, batch_first = False) : 
    tokens = tokenizer(quotes, max_length = 32, return_tensors = "pt", truncation=True, padding="max_length")
    
    if not batch_first:  
        tokens["input_ids"] = tokens["input_ids"].reshape(1, -1, 32)
        tokens["attention_mask"] = tokens["attention_mask"].reshape(1, -1, 32)
    else : 
        tokens["input_ids"] = tokens["input_ids"].reshape(-1, 1, 32)
        tokens["attention_mask"] = tokens["attention_mask"].reshape(-1, 1, 32)
        
    return tokens

In [ ]:
scores = []
aucs = []
labels, preds = [], []
model.eval()
with torch.no_grad() : 
    for nidx, novel_pairs in enumerate(pairs) : 
        
        scores_n = []
        labels_n, preds_n = [], []

        quotes = np.asarray( corpus["quotes"][nidx])
        for u, (speaker_id, anchor, neg_and_pos) in enumerate(novel_pairs) :
             
            q1 =quotes[anchor]
            tokens = tokenize(q1.tolist())
            ep1 = F.normalize( model(**tokens.to("cuda:0")), dim =1 ).cpu()
            
            q2 = quotes[neg_and_pos[speaker_id]]
            tokens = tokenize(q2.tolist(), batch_first = True)
            ep2 = F.normalize( model(**tokens.to("cuda:0")), dim =1 ).cpu()
            
            embeddings = [ep2]
            true_relevance = [1] * ep2.size(0)
            for sid, quote_ids in neg_and_pos.items() :
                if sid != speaker_id  : 
                    neg = quotes[quote_ids]
                    tokens = tokenize(neg.tolist(), batch_first = True)
                    eneg = F.normalize( model(**tokens.to("cuda:0")), dim =1 )                
                    embeddings.append(eneg.cpu())
                    true_relevance.extend([0] * eneg.size(0))
                    
            sims = util.dot_score(ep1[0], torch.cat(embeddings, dim = 0))[0]
            score = ndcg_score(np.asarray(true_relevance)[np.newaxis, :], sims.cpu().numpy()[np.newaxis, :])
            scores_n.append(score)
            labels_n.extend(true_relevance)
            preds_n.extend(sims.cpu().numpy() )
            
        scores.append(scores_n)
        aucs.append(roc_auc_score(labels_n, preds_n))
        labels.extend(labels_n)
        preds.extend(preds_n)
        
        print(f"NOVEL {nidx} ---- nDCG {np.mean(scores_n):0.3f} | AUC {aucs[-1]:0.3f}")
        
    overall_auc = roc_auc_score(labels, preds)
    print("")
    print(f"Overall nDCG {np.mean(sum(scores,[])):0.3f} | (average novel) nDCG {np.mean([np.mean(i) for i in scores]):0.3f} | AUC (average novels) {np.mean(aucs):0.3f} | Overall AUC {overall_auc:0.3f}")


# Explicit Quotes Author Verification (Representation)

In [ ]:
corpus = ExplicitQuoteCorpus("/data/datasets/project-dialogism-novel-corpus/data/")

In [ ]:
pairs = corpus.explicit_AV_samples()

## Semantics

In [ ]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')
quote_embeddings = []

for quotes in tqdm.tqdm(corpus["quotes"]) : 
    quote_embeddings.append(model.encode(quotes, device=torch.device("cuda:0"), normalize_embeddings=True, convert_to_numpy=False, convert_to_tensor=True))

In [ ]:
overall_auc, aucs, recalls = score(quote_embeddings, pairs)

## STEL

In [ ]:
model = SentenceTransformer('AnnaWegmann/Style-Embedding')
quote_embeddings = []

for quotes in tqdm.tqdm(corpus["quotes"]) : 
    quote_embeddings.append(model.encode(quotes, device=torch.device("cuda:0"), normalize_embeddings=True, convert_to_numpy=False, convert_to_tensor=True))

In [ ]:
overall_auc, aucs, recalls = score(quote_embeddings, pairs)

## EmotionS

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('SamLowe/roberta-base-go_emotions')
config = AutoConfig.from_pretrained("SamLowe/roberta-base-go_emotions")
model = AutoModelForSequenceClassification.from_pretrained("SamLowe/roberta-base-go_emotions", config=config).to("cuda:0")


quote_embeddings = []

batch_size = 16
with torch.no_grad() : 
    for idx, quotes in tqdm.tqdm(enumerate(corpus["quotes"])) : 
        last_h = []
        
        model.eval()
        for idx in range(0,len(quotes), batch_size) : 
            tokens = tokenizer(quotes[idx:idx+batch_size], return_tensors="pt", truncation=True, max_length=512, padding=True)
            u = model(**tokens.to("cuda:0"), return_dict=True, output_hidden_states = True)
            last_h.append(u.hidden_states[-1][:,0].detach().cpu())
        quote_embeddings.append(F.normalize(torch.cat(last_h), dim = -1))

In [ ]:
overall_auc, aucs, recalls = score(quote_embeddings, pairs)

## LUAR

In [ ]:
#device = "cpu"
tokenizer = AutoTokenizer.from_pretrained("rrivera1849/LUAR-MUD", trust_remote_code=True)
model = AutoModel.from_pretrained("rrivera1849/LUAR-MUD", trust_remote_code=True).to("cuda:0")
def tokenize(quotes) : 
    tokens = tokenizer(quotes, max_length = 32, return_tensors = "pt", truncation=True, padding="max_length")
    tokens["input_ids"] = tokens["input_ids"].reshape(1, -1, 32)
    tokens["attention_mask"] = tokens["attention_mask"].reshape(1, -1, 32)
    
    return tokens

In [ ]:
labels = []
preds = []
aucs = []
R1 = []
R3 = []
R8 = []

model.eval()
with torch.no_grad() : 
    for nidx, novel_pairs in enumerate(pairs) : 
        
        labels_n = []
        preds_n = []
        r_1, r_3, r_8 = [], [], []
        quotes = np.asarray( corpus["quotes"][nidx])
        for u, (speaker_id, anchor, neg_and_pos) in enumerate(novel_pairs) :
             
            q1 =quotes[anchor]
            tokens = tokenize(q1.tolist())
            ep1 = F.normalize( model(**tokens.to("cuda:0")), dim =1 ).cpu()
            
            q2 = quotes[neg_and_pos[speaker_id]]
            tokens = tokenize(q2.tolist())
            ep2 = F.normalize( model(**tokens.to("cuda:0")), dim =1 ).cpu()
            
            embeddings = [ep2]
            for sid, quote_ids in neg_and_pos.items() :
                if sid != speaker_id  : 
                    neg = quotes[quote_ids]
                    tokens = tokenize(neg.tolist())
                    eneg = F.normalize( model(**tokens.to("cuda:0")), dim =1 )                
                    embeddings.append(eneg.cpu())
                    
            sims = util.dot_score(ep1[0], torch.cat(embeddings, dim = 0))[0]
            sorted_idx = torch.argsort(sims)
            r_1.append(1 if 0 in sorted_idx[-1:] else 0 )
            r_3.append(1 if 0 in sorted_idx[-3:] else 0 )
            r_8.append(1 if 0 in sorted_idx[-8:] else 0 )

            labels_n.extend([1] + [0] * (len(sims) - 1))
            preds_n.extend(sims.tolist())    
            
        R8.append(sum(r_8) / len(novel_pairs))
        R1.append(sum(r_1) / len(novel_pairs))
        R3.append(sum(r_3) / len(novel_pairs))

        aucs.append(roc_auc_score(labels_n, preds_n))
        labels.extend(labels_n)
        preds.extend(preds_n)
        print(f"NOVEL {nidx} ---- Recall@{1} {R1[-1]:0.3f} | Recall@{3} {R3[-1]:0.3f} | Recall@{8} {R8[-1]:0.3f} | AUC {aucs[-1]:0.3f}")

    overall_auc = roc_auc_score(labels, preds)
    print("")
    print(f"Recall@{1} {np.mean(R1):0.3f} | Recall@{3} {np.mean(R3):0.3f} | Recall@{8} {np.mean(R8):0.3f} | AUC (average novels) {np.mean(aucs):0.3f} | Overall AUC {overall_auc:0.3f}")

# Explicit Quotes Author Verification (Quote per quote)

In [ ]:
corpus = ExplicitQuoteCorpus("/data/datasets/project-dialogism-novel-corpus/data/")

In [ ]:
pairs = corpus.explicit_AV_samples()

## Semantics

In [ ]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')
quote_embeddings = []

for quotes in tqdm.tqdm(corpus["quotes"]) : 
    quote_embeddings.append(model.encode(quotes, device=torch.device("cuda:0"), normalize_embeddings=True, convert_to_numpy=False, convert_to_tensor=True))

In [ ]:
scores = score_quote_by_quote(quote_embeddings, pairs, simulate_randomness=True)

## STEL

In [ ]:
model = SentenceTransformer('AnnaWegmann/Style-Embedding')
quote_embeddings = []

for quotes in tqdm.tqdm(corpus["quotes"]) : 
    quote_embeddings.append(model.encode(quotes, device=torch.device("cuda:0"), normalize_embeddings=True, convert_to_numpy=False, convert_to_tensor=True))

In [ ]:
scores = score_quote_by_quote(quote_embeddings, pairs)

## Emotions

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('SamLowe/roberta-base-go_emotions')
config = AutoConfig.from_pretrained("SamLowe/roberta-base-go_emotions")
model = AutoModelForSequenceClassification.from_pretrained("SamLowe/roberta-base-go_emotions", config=config).to("cuda:0")


quote_embeddings = []

batch_size = 16
with torch.no_grad() : 
    for idx, quotes in tqdm.tqdm(enumerate(corpus["quotes"])) : 
        last_h = []
        
        model.eval()
        for idx in range(0,len(quotes), batch_size) : 
            tokens = tokenizer(quotes[idx:idx+batch_size], return_tensors="pt", truncation=True, max_length=512, padding=True)
            u = model(**tokens.to("cuda:0"), return_dict=True, output_hidden_states = True)
            last_h.append(u.hidden_states[-1][:,0].detach().cpu())
        quote_embeddings.append(F.normalize(torch.cat(last_h), dim = -1))

In [ ]:
scores = score_quote_by_quote(quote_embeddings, pairs)

## LUAR

In [ ]:
#device = "cpu"
tokenizer = AutoTokenizer.from_pretrained("rrivera1849/LUAR-MUD", trust_remote_code=True)
model = AutoModel.from_pretrained("rrivera1849/LUAR-MUD", trust_remote_code=True).to("cuda:0")


def tokenize(quotes, batch_first = False) : 
    tokens = tokenizer(quotes, max_length = 32, return_tensors = "pt", truncation=True, padding="max_length")
    
    if not batch_first:  
        tokens["input_ids"] = tokens["input_ids"].reshape(1, -1, 32)
        tokens["attention_mask"] = tokens["attention_mask"].reshape(1, -1, 32)
    else : 
        tokens["input_ids"] = tokens["input_ids"].reshape(-1, 1, 32)
        tokens["attention_mask"] = tokens["attention_mask"].reshape(-1, 1, 32)
        
    return tokens

In [ ]:
scores = []
aucs = []
labels, preds = [], []
model.eval()
with torch.no_grad() : 
    for nidx, novel_pairs in enumerate(pairs) : 
        
        scores_n = []
        labels_n, preds_n = [], []

        quotes = np.asarray( corpus["quotes"][nidx])
        for u, (speaker_id, anchor, neg_and_pos) in enumerate(novel_pairs) :
             
            q1 =quotes[anchor]
            tokens = tokenize(q1.tolist())
            ep1 = F.normalize( model(**tokens.to("cuda:0")), dim =1 ).cpu()
            
            q2 = quotes[neg_and_pos[speaker_id]]
            tokens = tokenize(q2.tolist(), batch_first = True)
            ep2 = F.normalize( model(**tokens.to("cuda:0")), dim =1 ).cpu()
            
            embeddings = [ep2]
            true_relevance = [1] * ep2.size(0)
            for sid, quote_ids in neg_and_pos.items() :
                if sid != speaker_id  : 
                    neg = quotes[quote_ids]
                    tokens = tokenize(neg.tolist(), batch_first = True)
                    eneg = F.normalize( model(**tokens.to("cuda:0")), dim =1 )                
                    embeddings.append(eneg.cpu())
                    true_relevance.extend([0] * eneg.size(0))
                    
            sims = util.dot_score(ep1[0], torch.cat(embeddings, dim = 0))[0]
            score = ndcg_score(np.asarray(true_relevance)[np.newaxis, :], sims.cpu().numpy()[np.newaxis, :])
            scores_n.append(score)
            labels_n.extend(true_relevance)
            preds_n.extend(sims.cpu().numpy() )
            
        scores.append(scores_n)
        aucs.append(roc_auc_score(labels_n, preds_n))
        labels.extend(labels_n)
        preds.extend(preds_n)
        
        print(f"NOVEL {nidx} ---- nDCG {np.mean(scores_n):0.3f} | AUC {aucs[-1]:0.3f}")
        
    overall_auc = roc_auc_score(labels, preds)
    print("")
    print(f"Overall nDCG {np.mean(sum(scores,[])):0.3f} | (average novel) nDCG {np.mean([np.mean(i) for i in scores]):0.3f} | AUC (average novels) {np.mean(aucs):0.3f} | Overall AUC {overall_auc:0.3f}")


# Varying Input Size - Reading Order (Representations)

In [5]:
corpus[0].speaker_id_by_chapter[0]

array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5])

In [4]:
ns = [0.1, 0.15, 0.2, 0.3, 0.5]
all_pairs = []
for n in ns : 
    print("*"*10 + f" PROCESSING CHAPTER PERCENT {n }" + "*"*10)
    pairs = corpus.chapterwise_AV_samples(percent_train_chapters=n, percent_test_chapters=0.5)
    all_pairs.append(pairs)

********** PROCESSING CHAPTER PERCENT 0.1**********
7 1 4.0
5
NOVEL: AHandfulOfDust, ID 0 Percent Active Speakers 0.059


KeyboardInterrupt: 

In [16]:
all_pairs[0][0]

[(5,
  array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
         17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]),
  defaultdict(list,
              {5: [1614,
                1615,
                1616,
                1617,
                1618,
                1619,
                1620,
                1621,
                1622,
                1623,
                1625,
                1627,
                1629,
                1631,
                1633,
                1635,
                1637,
                1656,
                1657,
                1658,
                1659,
                1713,
                1715,
                1717,
                1719,
                1721,
                1723,
                1725,
                1727,
                1729],
               7: [1574,
                1575,
                1576,
                1577,
                1578,
                1624,
                1626,
                16

In [16]:
s_id = 1 
p10 = []
p50 = []


p10 = [all_pairs[0][0][i] for i in range(len(all_pairs[0][0])) if all_pairs[0][0][i][0] == 5]
p50 = [all_pairs[0][-1][i] for i in range(len(all_pairs[0][-1])) if all_pairs[0][-1][i][0] == 5]

In [23]:
p10[0][1]

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28]

In [25]:
p50[0]

(5,
 [528,
  530,
  532,
  534,
  536,
  539,
  544,
  547,
  549,
  551,
  560,
  562,
  564,
  569,
  570,
  572,
  574,
  576,
  578,
  580,
  582,
  584,
  586,
  587,
  590,
  593],
 defaultdict(list,
             {5: [618, 658, 699, 701, 819, 825],
              1: [460, 462, 464, 466, 844, 846, 848, 850, 851, 853],
              2: [144,
               147,
               149,
               152,
               154,
               156,
               159,
               161,
               164,
               165,
               193,
               196,
               256,
               258,
               262,
               315,
               317,
               319,
               321,
               323,
               325,
               468,
               589,
               592,
               594,
               603,
               605,
               607,
               609,
               611,
               614,
               616,
               642,
             

## Semantics

In [ ]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')
quote_embeddings = []

for quotes in tqdm.tqdm(corpus["quotes"]) : 
    quote_embeddings.append(model.encode(quotes, device=torch.device("cuda:0"), normalize_embeddings=True, convert_to_numpy=False, convert_to_tensor=True).cpu())

In [ ]:
semantic_scores = []

for pairs in all_pairs : 
    
    overall_auc, aucs, recalls = score(quote_embeddings, pairs)
    semantic_scores.append((overall_auc, aucs, recalls))

In [ ]:
quoted_semantic_scores = []

for pairs in all_pairs : 
    
    overall_auc, aucs, recalls = score_quote_by_quote(quote_embeddings, pairs)
    quoted_semantic_scores.append((overall_auc, aucs, recalls))

## STEL

In [ ]:
model = SentenceTransformer('AnnaWegmann/Style-Embedding')
quote_embeddings = []

for quotes in tqdm.tqdm(corpus["quotes"]) : 
    quote_embeddings.append(model.encode(quotes, device=torch.device("cuda:0"), normalize_embeddings=True, convert_to_numpy=False, convert_to_tensor=True))

In [ ]:
stel_scores = []

for pairs in all_pairs : 
    
    overall_auc, aucs, recalls = score(quote_embeddings, pairs)
    stel_scores.append((overall_auc, aucs, recalls))

In [ ]:
quoted_stel_scores = []

for pairs in all_pairs : 
    overall_auc, aucs, recalls = score_quote_by_quote(quote_embeddings, pairs)
    quoted_stel_scores.append((overall_auc, aucs, recalls))

## Emotions

In [ ]:


tokenizer = AutoTokenizer.from_pretrained('SamLowe/roberta-base-go_emotions')
config = AutoConfig.from_pretrained("SamLowe/roberta-base-go_emotions")
model = AutoModelForSequenceClassification.from_pretrained("SamLowe/roberta-base-go_emotions", config=config).to("cuda:0")



quote_embeddings = []

batch_size = 32
with torch.no_grad() : 
    for idx, quotes in tqdm.tqdm(enumerate(corpus["quotes"])) : 
        last_h = []
        
        model.eval()
        for idx in range(0,len(quotes), batch_size) : 
            tokens = tokenizer(quotes[idx:idx+batch_size], return_tensors="pt", truncation=True, max_length=512, padding=True)
            u = model(**tokens.to("cuda:0"), return_dict=True, output_hidden_states = True)
            last_h.append(u.hidden_states[-1][:,0].detach().cpu())
        quote_embeddings.append(F.normalize(torch.cat(last_h), dim = -1))

In [ ]:
emotion_scores = []

for pairs in all_pairs : 
    
    overall_auc, aucs, recalls = score(quote_embeddings, pairs)
    emotion_scores.append((overall_auc, aucs, recalls))

In [ ]:
quoted_emotion_scores = []

for pairs in all_pairs : 
    
    overall_auc, aucs, recalls = score_quote_by_quote(quote_embeddings, pairs)
    quoted_emotion_scores.append((overall_auc, aucs, recalls))

## LUAR

In [ ]:
!pip install einops

In [ ]:
#device = "cpu"
tokenizer = AutoTokenizer.from_pretrained("rrivera1849/LUAR-MUD", trust_remote_code=True)
model = AutoModel.from_pretrained("rrivera1849/LUAR-MUD", trust_remote_code=True).to("cuda:0")
def tokenize(quotes, batch_first = False) : 
    tokens = tokenizer(quotes, max_length = 32, return_tensors = "pt", truncation=True, padding="max_length")
    
    if not batch_first:  
        tokens["input_ids"] = tokens["input_ids"].reshape(1, -1, 32)
        tokens["attention_mask"] = tokens["attention_mask"].reshape(1, -1, 32)
    else : 
        tokens["input_ids"] = tokens["input_ids"].reshape(-1, 1, 32)
        tokens["attention_mask"] = tokens["attention_mask"].reshape(-1, 1, 32)
        
    return tokens
    

In [ ]:

def score(quote_embeddings, pairs):
    scores = defaultdict(list)
    labels = []
    preds = []

    for nidx, novel_pairs in enumerate(pairs):
        if len(novel_pairs) == 0:
            print(f"NOVEL {nidx} ---- Found no pairs")
            scores["aucs_mean"].append(None)
            scores["aucs_std"].append(None)

            scores["R1"].append(None)
            scores["R3"].append(None)
            scores["R8"].append(None)

        else:
            labels_n = []
            preds_n = []
            speaker_aucs = []
            r_1, r_3, r_8 = [], [], []

            q_emb = quote_embeddings[nidx]
            for u, (speaker_id, anchor, neg_and_pos) in enumerate(novel_pairs):
                ep1 = q_emb[anchor].mean(dim=0)
                ep2 = q_emb[neg_and_pos[speaker_id]].mean(dim=0)

                embeddings = [ep2.unsqueeze(0)]
                for sid, quote_ids in neg_and_pos.items():
                    if sid != speaker_id:
                        eneg = q_emb[quote_ids].mean(dim=0)
                        embeddings.append(eneg.unsqueeze(0))

                sims = util.dot_score(ep1, torch.cat(embeddings, dim=0))[0]
                sorted_idx = torch.argsort(sims)
                r_1.append(1 if 0 in sorted_idx[-1:] else 0)
                r_3.append(1 if 0 in sorted_idx[-3:] else 0)
                r_8.append(1 if 0 in sorted_idx[-8:] else 0)

                speaker_label = [1] + [0] * (len(sims) - 1)
                speaker_pred = sims.tolist()
                speaker_aucs.append(roc_auc_score(speaker_label, speaker_pred))

                labels_n.extend()
                preds_n.extend(sims.tolist())

            scores["R8"].append(sum(r_8) / len(novel_pairs))
            scores["R1"].append(sum(r_1) / len(novel_pairs))
            scores["R3"].append(sum(r_3) / len(novel_pairs))

            scores["aucs_mean"].append(np.mean(speaker_aucs))
            scores["aucs_std"].append(np.std(speaker_aucs))

            labels.extend(labels_n)
            preds.extend(preds_n)
            print(
                f"NOVEL {nidx} ---- Recall@{1} {scores['R1'][-1]:0.3f} | Recall@{3} {scores['R3'][-1]:0.3f} | Recall@{8} {scores['R8'][-1]:0.3f} | AUC {scores['auc_mean'][-1]:0.3f}"
            )

    overall_auc = roc_auc_score(labels, preds)

    mean_aucs = np.mean([i for i in scores["auc_mean"] if i != None])
    mean_R1 = np.mean([i for i in scores["R1"] if i != None])
    mean_R3 = np.mean([i for i in scores["R3"] if i != None])
    mean_R8 = np.mean([i for i in scores["R8"] if i != None])

    print("")
    print(
        f"Recall@{1} {mean_R1:0.3f} | Recall@{3} {mean_R3:0.3f} | Recall@{8} {mean_R8:0.3f} | AUC (average novels) {mean_aucs:0.3f} | Overall AUC {overall_auc:0.3f}"
    )

    return scores

In [ ]:
def luar_score(model, quotes, pairs) : 
    scores = defaultdict(list)
    model.eval()
    with torch.no_grad() : 
        for nidx, novel_pairs in enumerate(pairs) : 
            if len(novel_pairs) == 0:
                print(f"NOVEL {nidx} ---- Found no pairs")
                scores["aucs_mean"].append(None)
                scores["aucs_std"].append(None)

                scores["R1"].append(None)
                scores["R3"].append(None)
                scores["R8"].append(None)
            else : 
                r_1, r_3, r_8 = [], [], []
                speaker_aucs = []
                quotes = np.asarray( corpus["quotes"][nidx])
                for u, (speaker_id, anchor, neg_and_pos) in enumerate(novel_pairs) :
                    
                    q1 =quotes[anchor]
                    tokens = tokenize(q1.tolist())
                    ep1 = F.normalize( model(**tokens.to("cuda:0")), dim =1 ).cpu()
                    
                    q2 = quotes[neg_and_pos[speaker_id]]
                    tokens = tokenize(q2.tolist())
                    ep2 = F.normalize( model(**tokens.to("cuda:0")), dim =1 ).cpu()
                    
                    embeddings = [ep2]
                    for sid, quote_ids in neg_and_pos.items() :
                        if sid != speaker_id  : 
                            neg = quotes[quote_ids]
                            tokens = tokenize(neg.tolist())
                            eneg = F.normalize( model(**tokens.to("cuda:0")), dim =1 )                
                            embeddings.append(eneg.cpu())
                            
                    sims = util.dot_score(ep1[0], torch.cat(embeddings, dim = 0))[0]
                    sorted_idx = torch.argsort(sims)
                    r_1.append(1 if 0 in sorted_idx[-1:] else 0 )
                    r_3.append(1 if 0 in sorted_idx[-3:] else 0 )
                    r_8.append(1 if 0 in sorted_idx[-8:] else 0 )

                    speaker_label = [1] + [0] * (len(sims) - 1)
                    speaker_pred = sims.tolist()
                    speaker_aucs.append(roc_auc_score(speaker_label, speaker_pred))

                scores["R8"].append(sum(r_8) / len(novel_pairs))
                scores["R1"].append(sum(r_1) / len(novel_pairs))
                scores["R3"].append(sum(r_3) / len(novel_pairs))

                scores["aucs_mean"].append(np.mean(speaker_aucs))
                scores["aucs_std"].append(np.std(speaker_aucs))
            
                print(
                    f"NOVEL {nidx} ---- Recall@{1} {scores['R1'][-1]:0.3f} | Recall@{3} {scores['R3'][-1]:0.3f} | Recall@{8} {scores['R8'][-1]:0.3f} | AUC {scores['auc_mean'][-1]:0.3f}"
                )
        mean_aucs = np.mean([i for i in aucs if i != None])
        mean_R1 = np.mean([i for i in R1 if i != None])
        mean_R3 = np.mean([i for i in R3 if i != None])
        mean_R8 =  np.mean([i for i in R8 if i != None])

        print("")
        print(f"Recall@{1} {mean_R1:0.3f} | Recall@{3} {mean_R3:0.3f} | Recall@{8} {mean_R8:0.3f} | AUC (average novels) {mean_aucs:0.3f}")

In [ ]:
def luar_score_quote_by_quote(model, tokenizer, corpus, pairs) : 
    scores = defaultdict(list)
    model.eval()
    with torch.no_grad() : 
        for nidx, novel_pairs in enumerate(pairs) : 
            if len(novel_pairs) == 0 : 
                print(f"NOVEL {nidx} ---- Found no pairs")
                scores["aucs_mean"].append(None)
                scores["aucs_std"].append(None)
                scores["ndcg_mean"].append(None)
                scores["ndcg_std"].append(None)                
            else : 
                scores_n = []
                speaker_aucs = []
                quotes = np.asarray( corpus["quotes"][nidx])
                for u, (speaker_id, anchor, neg_and_pos) in enumerate(novel_pairs) :
                    
                    q1 =quotes[anchor]
                    tokens = tokenize(tokenizer, q1.tolist())
                    ep1 = F.normalize( model(**tokens.to("cuda:0")), dim =1 ).cpu()
                    
                    q2 = quotes[neg_and_pos[speaker_id]]
                    tokens = tokenize(tokenizer, q2.tolist(), batch_first = True)
                    ep2 = F.normalize( model(**tokens.to("cuda:0")), dim =1 ).cpu()
                    
                    embeddings = [ep2]
                    true_relevance = [1] * ep2.size(0)
                    for sid, quote_ids in neg_and_pos.items() :
                        if sid != speaker_id  : 
                            neg = quotes[quote_ids]
                            tokens = tokenize(tokenizer, neg.tolist(), batch_first = True)
                            eneg = F.normalize( model(**tokens.to("cuda:0")), dim =1 )                
                            embeddings.append(eneg.cpu())
                            true_relevance.extend([0] * eneg.size(0))
                            
                    sims = util.dot_score(ep1[0], torch.cat(embeddings, dim = 0))[0]
                    
                    score = ndcg_score(np.asarray(true_relevance)[np.newaxis, :], sims.cpu().numpy()[np.newaxis, :])
                    scores_n.append(score)
                    speaker_aucs.append(roc_auc_score(true_relevance, sims.cpu().numpy()))

                scores["ndcg_mean"].append(np.mean(scores_n))
                scores["ndcg_std"].append(np.std(scores_n))
                scores["auc_mean"].append(np.mean(speaker_aucs))
                scores["auc_std"].append(np.std(speaker_aucs))
                
                print(
                    f"NOVEL {nidx} ---- nDCG {scores['ndcg_mean'][-1]:0.3f} | AUC {scores['auc_mean'][-1]:0.3f}"
                )
            
        mean_aucs = np.mean([i for i in scores["auc_mean"] if i != None])
        mean_scores = np.mean([i for i in scores["ndcg_mean"] if i is not None])

        print("")
        print(
                    f"(average novel) nDCG {mean_scores:0.3f} | AUC (average novels) {mean_aucs:0.3f}"
                )
        return scores        


# Plots

## Character - Representation

In [ ]:
def mean(l) :
    return np.mean([i for i in l if i is not None])

for model_name, scores in zip(["Semantics", "STEL", "Emotion", "LUAR"], [semantic_scores, stel_scores, emotion_scores, luar_scores] ): 
    aucs = []
    for n, score_ in zip(ns, scores) : 
        aucs.append(mean(score_[1]))
    plt.plot(ns, aucs, marker = ".", label= model_name, alpha=0.8, lw=2)
    
#plt.plot(ns, R1s, marker = ".", label = "Recall@1", color="green", alpha=0.7, lw=2)
plt.xlim(0.05, 0.55)
plt.title("Character - Character")
plt.xlabel(ns)
#plt.ylim(min(aucs)-0.05, max(aucs)+0.05)
plt.legend(frameon=True)
plt.grid()
plt.ylabel("AUC")
plt.xlabel("% of chapters used for encoding")

In [ ]:
for model_name, scores in zip(["Semantics", "STEL", "Emotion", "LUAR"], [semantic_scores, stel_scores, emotion_scores, luar_scores] ): 
    r1s= []
    for n, score_ in zip(ns, scores) : 
        r1s.append(mean(score_[2][0]))
    plt.plot(ns, r1s, marker = ".", label= model_name, alpha=0.8, lw=2)
    
#plt.plot(ns, R1s, marker = ".", label = "Recall@1", color="green", alpha=0.7, lw=2)
plt.xlim(0.05, 0.55)
plt.title("Character - Character")
plt.xlabel(ns)
#plt.ylim(min(aucs)-0.05, max(aucs)+0.05)
plt.legend(frameon=True)
plt.grid()
plt.ylabel("Recall@1")
plt.xlabel("% of chapters used for encoding")

## Character - Quotes

In [ ]:
for model_name, scores in zip(["Semantics", "STEL", "Emotion", "LUAR"], [quoted_semantic_scores, quoted_stel_scores, quoted_emotion_scores, quoted_luar_scores] ): 
    aucs = []
    for n, score_ in zip(ns, scores) : 
        aucs.append(mean(score_[1]))
    plt.plot(ns, aucs, marker = ".", label= model_name, alpha=0.8, lw=2)
    
#plt.plot(ns, R1s, marker = ".", label = "Recall@1", color="green", alpha=0.7, lw=2)
plt.xlim(0.05, 0.55)
plt.title("Character - Quote")
plt.xlabel(ns)
#plt.ylim(min(aucs)-0.05, max(aucs)+0.05)
plt.legend(frameon=True)
plt.grid()
plt.ylabel("AUC")
plt.xlabel("Chapter %")